# Data loading

In [3]:
!pip install -q fastplot

## Project Zero and CISA KEV

In [ ]:
# Filter CVEs in PZ and KEV that reached T in 2014-2023 and in 2022/2023 and save the relative vendor, product and type
def retrieve_cve_by_epss_above_threshold(df):
    df = df.groupby('cve')['epss'].apply(lambda l: l.values.max())
    return df.groupby('cve').filter(lambda v: (v > THRESHOLD).any()).index.values


def filter_df_by_cve_list(df, cve_column_name, cve_list):
    return df[df[cve_column_name].isin(cve_list)]


df_pz_sheet_filtered = df_pz_sheet[['CVE', 'Vendor', 'Product', 'Type']].dropna()
highest_cve_pz = retrieve_cve_by_epss_above_threshold(df_pz)
df_pz_sheet_highest = filter_df_by_cve_list(df_pz_sheet_filtered, 'CVE', highest_cve_pz)

df_kev_sheet_filtered = df_kev_sheet[['cveID', 'vendorProject', 'product', 'vulnerabilityName']]
highest_cve_kev = retrieve_cve_by_epss_above_threshold(df_kev)
df_kev_sheet_highest = filter_df_by_cve_list(df_kev_sheet_filtered, 'cveID', highest_cve_kev)

### Distribution of CVEs that exceed the threshold vs vendor, product, type of vulnerability for Project Zero

In [ ]:
# Number of CVE of pz with EPSS > T per vendor
vendors = list(df_pz_sheet_highest['Vendor'].unique())
data = [(v, len(df_pz_sheet_highest[df_pz_sheet_highest['Vendor'] == v])) for v in vendors]
fastplot.plot(data, None, mode='bars', xlabel='Vendor', ylabel='#CVE 0day with EPSS > T (>= 1 day)',
              xticks_rotate=30, figsize=(8, 4), grid=True)

In [ ]:
# Number of CVE of pz with EPSS > T per product
products = list(df_pz_sheet_highest['Product'].unique())
data = [(p[:10], len(df_pz_sheet_highest[df_pz_sheet_highest['Product'] == p])) for p in products]
fastplot.plot(data, None, mode='bars', xlabel='Product affected', ylabel='#CVE 0day with EPSS > T (>= 1 day)',
              xticks_rotate=30, figsize=(8, 4), grid=True)

In [ ]:
# Number of CVE of pz with EPSS > T per type of product
types = list(df_pz_sheet_highest['Type'].unique())
data = [(t[:10], len(df_pz_sheet_highest[df_pz_sheet_highest['Type'] == t])) for t in types]
fastplot.plot(data, None, mode='bars', xlabel='Type of vulnerability',
              ylabel='#CVE 0day with EPSS > T (>= 1 day)', figsize=(8, 4), grid=True)

### Distribution of CVEs that exceed the threshold vs vendor, product for CISA KEV

In [2]:
def group_values_with_less_frequency(tuples_list, keep_other=False):
    sum_of_ones = 0
    other_tuples = []
    for string, num in tuples_list:
        if num == 1:
            sum_of_ones += 1
        else:
            other_tuples.append((string, num))
    if sum_of_ones > 0 and keep_other == True:
        other_tuples.append(('Other', sum_of_ones))
    return other_tuples

In [ ]:
# Number of CVE of KEV with EPSS > T per vendor
vendors = list(df_kev_sheet_highest['vendorProject'].unique())
data = [(v, len(df_kev_sheet_highest[df_kev_sheet_highest['vendorProject'] == v])) for v in vendors]
data = group_values_with_less_frequency(data)
fastplot.plot(data, None, mode='bars', xlabel='Vendor', ylabel='#CVE KEV with EPSS > T (>= 1 day)',
              xticks_rotate=90, figsize=(12, 6), grid=True)

In [ ]:
# Number of CVE of KEV with EPSS > T per product
products = list(df_kev_sheet_highest['product'].unique())
data = [(p[:10], len(df_kev_sheet_highest[df_kev_sheet_highest['product'] == p])) for p in products]
data = group_values_with_less_frequency(data)
fastplot.plot(data, None, mode='bars', xlabel='Product', ylabel='#CVE KEV with EPSS > T (>= 1 day)',
              xticks_rotate=90, figsize=(12, 6), grid=True)

## hackthebox - Most popular CVEs from 2022

In [ ]:
# Extract CVEs list from the website
data = requests.get('https://www.hackthebox.com/blog/most-popular-cybersecurity-vulnerabilities-and-exploits-from-2022')
soup = BeautifulSoup(data.text, 'html.parser')
table_cells = soup.find_all('td')
cve_hackthebox = []
for cell in table_cells:
    paragraphs = cell.find_all('p')
    for paragraph in paragraphs:
        links = paragraph.find_all('a')
        for link in links:
            cve_id = link.text
            if 'cve' in cve_id:
                cve_hackthebox.append(cve_id.upper())

In [ ]:
# How many of those are inside the other dfs?
highest_cve_2223 = compute_highest_cve_list_2223(df_highest_epss)
cve_hackthebox_highest_2223 = list(set(highest_cve_2223) & set(cve_hackthebox))
cve_hackthebox_pz_2223 = list(set(highest_cve_pz) & set(cve_hackthebox))
cve_hackthebox_kev_2223 = list(set(highest_cve_kev) & set(cve_hackthebox))
print(f"CVEs from hackthebox inside df_highest_epss_2223: {len(cve_hackthebox_highest_2223)}")
print(f"CVEs from hackthebox inside df_pz: {len(cve_hackthebox_pz_2223)}")
print(f"CVEs from hackthebox inside df_kev: {len(cve_hackthebox_kev_2223)}")